In [1]:
# libraries
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math, time

# local functions
from utils import import_matrix

## Loading data

In [2]:
root    = './'
city    = 'test_dataset'

In [3]:
net_file = os.path.join(root, city, 'net.csv')
network = pd.read_csv(net_file, sep=',')

trimmed_cols = [s.strip().lower() for s in network.columns]
network.columns = trimmed_cols

In [4]:
edge_attr = {}
for row in network.itertuples():
    edge_attr[(row[1], row[2])] = {'capacity':row[3], 'length':row[4], 'FFT':row[5], 'alpha':row[6],
                                    'beta':row[7], 'type':row[10], 'flow':0, 'cost':row[5], 'Oflow':None}
    
edge = [(row[1], row[2]) for row in network.itertuples()]

In [5]:
network_graph = nx.DiGraph()
network_graph.add_edges_from(edge)
nx.set_edge_attributes(network_graph, edge_attr)

In [6]:
origin_destination_raw = pd.read_csv(os.path.join(root, city, 'ODdemand.csv'), sep=',', header=0)

origin_destination_demand = {(int(row['Origin']), int(row['Destination'])): int(row['Demand']) for index, row in origin_destination_raw.iterrows()}
# origin_destination_demand

In [7]:
centroid_data = pd.read_csv(os.path.join(root, city, 'centroid.csv'), sep=',')
zones2centroid = centroid_data.groupby('fishnetID')['osmid'].apply(list).to_dict()
# zones2centroid

## All or nothing assignment

In [8]:
def AONloading(graph, zone2centroid, demand, compute_sptt=True):
    """
    All-or-Nothing (AON) traffic assignment with shortest path travel time (SPTT) computation.

    Parameters:
        graph (nx.DiGraph): A directed graph representing the road network, with 'cost' as the edge weight.
        zone2centroid (dict): Maps zones to lists of centroid nodes within each zone.
        demand (dict): OD demand values as a dictionary {(origin_zone, destination_zone): demand}.
        compute_sptt (bool): Flag to compute SPTT and EODTT for OD pairs.

    Returns:
        tuple: 
            - SPTT (float): Total shortest path travel time across all OD pairs.
            - x_bar (dict): Edge flow for each edge in the graph.
            - spedges (dict): Shortest paths for each OD pair (optional, if `compute_sptt` is True).
            - EODTT (dict): End-to-end travel times for OD pairs (optional, if `compute_sptt` is True).
    """
    # Initialize outputs
    x_bar = {edge: 0 for edge in graph.edges()}  # Edge flows
    spedges = {}  # Shortest paths for OD pairs
    EODTT = {}  # End-to-end travel times
    SPTT = 0  # Shortest path travel time
    
    # Iterate through origin zones and their centroid nodes
    for origin_zone, origin_nodes in zone2centroid.items():
        # Compute shortest paths from all centroids in the origin zone
        dijkstra_results = [
            nx.single_source_dijkstra(graph, origin_node, weight="cost") for origin_node in origin_nodes
        ]
        
        # Iterate through destination zones and their centroid nodes
        for destination_zone, destination_nodes in zone2centroid.items():
            if origin_zone == destination_zone:
                continue  # Skip intra-zone flows
            
            # Get the demand for the OD pair
            od_demand = demand.get((origin_zone, destination_zone), 0)
            if od_demand <= 0:
                continue  # Skip if no demand
            
            # Find the shortest path among all centroid-to-centroid paths
            shortest_paths = [
                (dijkstra_results[i][0][dest_node], dijkstra_results[i], dest_node)
                for i in range(len(dijkstra_results))
                for dest_node in destination_nodes
                if dest_node in dijkstra_results[i][0]
            ]
            
            if not shortest_paths:
                continue  # Skip if no valid path exists
            
            # Select the shortest path
            shortest_paths.sort(key=lambda x: x[0])  # Sort by travel time
            min_time, dijkstra_result, destination_node = shortest_paths[0]
            path = dijkstra_result[1][destination_node]

            # Compute SPTT and store path/EODTT if required
            if compute_sptt:
                SPTT += min_time * od_demand
                spedges[(origin_zone, destination_zone)] = path
                EODTT[(origin_zone, destination_zone)] = min_time
            
            # Update edge flows
            for u, v in zip(path[:-1], path[1:]):
                x_bar[(u, v)] += od_demand
    
    return SPTT, x_bar, spedges, EODTT


In [9]:
G_AON = network_graph.copy()
SPTT, x_bar, spedges, EODTT = AONloading(G_AON, zones2centroid, origin_destination_demand, compute_sptt=True)
print('SPTT: ', SPTT)
print('x_bar: ', x_bar)
print('spedges: ', spedges)
print('EODTT: ', EODTT)

SPTT:  165000
x_bar:  {(10, 11): 0, (12, 13): 3000, (14, 15): 0}
spedges:  {(0, 3): [12, 13]}
EODTT:  {(0, 3): 55}


## Task:
Develop the MSA assignment code that evaulates user equilibrium assignment given the SiouxFalls network and demand



## Work flow to develop MSA
Initial run
* Run dijkstra
* Run AON loading assignment (AON function)
* Update network costs/traveltimes
* Run Dijkstra
* Check relative gap between TSTT and SPTT - Gap is (AON volumes * new costs) / SPTT. SPTT is all demand * min(new costs)

Iterative
* Run AON again
* Calculate MSA volumes/flows
* Update network costs/traveltimes 
* Run dijkstra again
* Check relative gap between TSTT and SPTT - Gap is (MSA volumes * new costs) / SPTT. SPTT is all demand * min(new costs)

In [12]:
def MSA(graph, zone2centroid, demand, max_iterations=100, convergence_threshold=0.05):
    """
    Perform User Equilibrium (UE) Assignment using the Method of Successive Averages (MSA).

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network, with 'cost' as edge weights.
        zone2centroid (dict): Mapping of zones to lists of centroid nodes within each zone.
        demand (dict): OD demand as a dictionary {(origin_zone, destination_zone): demand}.
        max_iterations (int): Maximum number of iterations to run.
        convergence_threshold (float): Threshold for the relative gap to check convergence.

    Returns:
        dict: Final edge flows (volumes) for each edge as {(u, v): flow}.
        list: Convergence history of relative gaps.
    """
    # Initialize edge flows and cost attributes
    for u, v, data in graph.edges(data=True):
        data['flow'] = 0  # Initialize edge flows to zero
        if 'cost' not in data:
            raise ValueError("Graph edges must have an initial 'cost' attribute.")
    
    relative_gaps = []  # To track convergence

    # Step 1: Initial All-or-Nothing (AON) assignment
    print("Running initial AON assignment...")
    SPTT, x_bar, _, _ = AONloading(graph, zone2centroid, demand, compute_sptt=True)

    for iteration in range(1, max_iterations + 1):
        print(f"Iteration {iteration}...")

        # Update flows using MSA: x = x + (1 / iteration) * (x_bar - x)
        for (u, v) in graph.edges():
            graph[u][v]['flow'] = (
                graph[u][v]['flow'] + (1 / iteration) * (x_bar.get((u, v), 0) - graph[u][v]['flow'])
            )

        # Update travel times/costs on the network based on new flows
        update_edge_costs(graph)

        # Step 2: Run shortest path (Dijkstra's algorithm) and AON loading with updated costs
        SPTT, x_bar, _, _ = AONloading(graph, zone2centroid, demand, compute_sptt=True)
        # Step 3: Compute the relative gap
        TSTT = sum(graph[u][v]['flow'] * graph[u][v]['cost'] for u, v in graph.edges())

        relative_gap = abs(TSTT - SPTT) / SPTT
        relative_gaps.append(relative_gap)
        
        print(f"Iteration {iteration}: Relative Gap = {relative_gap:.6f} TSTT = {TSTT}")

        # Check for convergence (relative gap < 0.05)
        if relative_gap < convergence_threshold:
            print("Convergence achieved!")
            break

    # Fixed return statement
    return {(u, v): graph[u][v]['flow'] for u, v in graph.edges()}, relative_gaps, graph



def update_edge_costs(graph):
    """
    Update edge travel times (costs) based on current flows using the BPR (Bureau of Public Roads) function.

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network.

    Notes:
        The BPR function is defined as:
            cost = free_flow_time * (1 + alpha * (flow / capacity)^beta)
    """
    alpha = 0.5
    beta = 4
    for u, v, data in graph.edges(data=True):
        free_flow_time = data.get('FFT', 1)
        capacity = data.get('capacity', 1)
        flow = data['flow']
        data['cost'] = free_flow_time * (1 + alpha * (flow / capacity) ** beta)


In [11]:
G_AON = network_graph.copy()
final_flows, gaps, Assigned_graph = MSA(G_AON, zones2centroid, origin_destination_demand)

print("Final Edge Flows:", final_flows)
print("Relative Gaps:", gaps)

Running initial AON assignment...
Iteration 1...
Iteration 1: Relative Gap = 286.375000 TSTT = 51727500.0
Iteration 2...
Iteration 2: Relative Gap = 6.450521 TSTT = 2011640.625
Iteration 3...
Iteration 3: Relative Gap = 0.666681 TSTT = 450003.8580246914
Iteration 4...
Iteration 4: Relative Gap = 0.751511 TSTT = 477583.92333984375
Iteration 5...
Iteration 5: Relative Gap = 0.257164 TSTT = 311148.0
Iteration 6...
Iteration 6: Relative Gap = 0.666681 TSTT = 450003.8580246914
Iteration 7...
Iteration 7: Relative Gap = 0.479867 TSTT = 406303.92098530376
Iteration 8...
Iteration 8: Relative Gap = 0.126864 TSTT = 319483.1943511963
Iteration 9...
Iteration 9: Relative Gap = 0.060968 TSTT = 286461.4134024285
Iteration 10...
Iteration 10: Relative Gap = 0.257164 TSTT = 311148.0
Iteration 11...
Iteration 11: Relative Gap = 0.195719 TSTT = 337132.8492216752
Iteration 12...
Iteration 12: Relative Gap = 0.121063 TSTT = 302687.0727539063
Iteration 13...
Iteration 13: Relative Gap = 0.094938 TSTT = 31